# Tagging and Extraction Using OpenAI functions

In [4]:
####.  Please use VirttualEnv: LCEL_extracting
####.  Please use VirttualEnv: LCEL_extracting
####.  Please use VirttualEnv: LCEL_extracting

import os
from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv()) 
_ = load_dotenv(dotenv_path="/Users/maxzhao/.env") 
print(os.environ["OPENAI_API_KEY"]),
OPENAI_API_KEY=os.environ["OPENAI_API_KEY"]
AWS_ACCESS_KEY_ID=os.environ["AWS_ACCESS_KEY_ID"]
AWS_SECRECT_ACCESS_KEY=os.environ["AWS_SECRECT_ACCESS_KEY"]

sk-iiP1T6p4dKPxH4Hn3Y0D3B0gqPqPwEpdDeSRxxhgABT3BlbkFJPUq8Jb-fqOSXJzUtDVtpvdG34DEHTtS7eVr9MqQlcA


In [5]:
#os.environ.clear()

In [6]:
import boto3
import json

# Initialize a session using your AWS credentials
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRECT_ACCESS_KEY,
    region_name='us-east-1'  # e.g., 'us-west-1'
)

# Create an S3 client
s3 = session.client('s3')

# Your JSON object
data = {
    "name": "John Doe",
    "age": 30,
    "city": "New York"
}

# Convert JSON object to string
json_data = json.dumps(data)

# Define your bucket name and the desired object name (key)
bucket_name = 'gz5427'
object_name = 'llm_extract/test.json'

# Upload JSON string to S3
s3.put_object(Bucket=bucket_name, Key=object_name, Body=json_data)

print(f"Uploaded {object_name} to {bucket_name}")

Uploaded llm_extract/test.json to gz5427


In [7]:
!pip freeze | grep langchain
#langchain==0.3.0
#langchain-community==0.3.0
#langchain-core==0.3.5
#langchain-openai==0.2.0
#langchain-text-splitters==0.3.0

/bin/sh: pip: command not found


In [8]:
!pip freeze | grep openai
#langchain-openai==0.2.0
#openai==1.47.0

/bin/sh: pip: command not found


In [9]:
!pip freeze | grep pydantic
#pydantic==2.9.2
#pydantic-settings==2.5.2
#pydantic_core==2.23.4


/bin/sh: pip: command not found


## Extraction

Extraction is similar to tagging, but used for extracting multiple pieces of information.

## Doing it for real

We can apply tagging to a larger body of text.

For example, let's load this blog post and extract tag information from a sub-set of the text.

In [23]:
from langchain_openai import ChatOpenAI
from typing import List, Optional
from pydantic import BaseModel, Field
#from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

model = ChatOpenAI(temperature=0)

In [24]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
template = """You are ahelpful assistant. Please hlep me to answer the questions."""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "Please tell me a joke about this animal: {animal}")
])

chain=prompt | model | StrOutputParser()  

chain.invoke({"animal":"elephant"})


'Why did the elephant bring a suitcase to the jungle? \n\nBecause he wanted to pack his trunk!'

In [25]:
import os
os.environ['USER_AGENT'] = 'myagent'

In [26]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [27]:
doc = documents[0]

In [28]:
page_content = doc.page_content[:10000]

In [29]:
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    langugae: str = Field(description="Provide the languge that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

In [30]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [31]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

overview_tagging_function = [convert_to_openai_function(Overview)]
tagging_model = model.bind(functions=overview_tagging_function, function_call={"name":"Overview"})
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser(key_name="overview")
tagging_chain.invoke({"input": page_content})

{'summary': 'The article discusses building autonomous agents powered by LLM (large language model) as the core controller, focusing on planning, memory, and tool use components.',
 'langugae': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use, task decomposition, self-reflection'}

Now let's try to extract all papers mentioned in this article

In [32]:
class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str
    author: Optional[str]


class Info(BaseModel):
    """Information to extract"""
    papers: List[Paper]

In [33]:
paper_extraction_function = [convert_to_openai_function(Info)]
extraction_model = model.bind(functions=paper_extraction_function, function_call={"name":"Info"})

In [34]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [35]:
extraction_chain.invoke({"input": page_content})

[{'title': 'LLM Powered Autonomous Agents', 'author': 'Lilian Weng'}]

In [36]:
template = """A article will be passed to you. Extract from it all papers that are mentioned by this article. 

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

In [37]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [38]:
extraction_chain.invoke({"input": page_content})

[{'title': 'Chain of thought (CoT; Wei et al. 2022)', 'author': None},
 {'title': 'Tree of Thoughts (Yao et al. 2023)', 'author': None},
 {'title': 'LLM+P (Liu et al. 2023)', 'author': None},
 {'title': 'ReAct (Yao et al. 2023)', 'author': None},
 {'title': 'Reflexion (Shinn & Labash 2023)', 'author': None},
 {'title': 'Chain of Hindsight (CoH; Liu et al. 2023)', 'author': None},
 {'title': 'Algorithm Distillation (AD; Laskin et al. 2023)', 'author': None}]

In [39]:
extraction_chain.invoke({"input": "hi"})

[]

In [40]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [41]:
splits = text_splitter.split_text(doc.page_content)

In [42]:
len(splits)

15

In [43]:
from langchain.schema.runnable import RunnableLambda

In [44]:
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [45]:
chain = (
    RunnableLambda(lambda x: [{"input": doc} for doc in text_splitter.split_text(x)])
    | extraction_chain.map()
    | flatten
)

In [46]:
output=chain.invoke(doc.page_content)

## Store data into s3.


In [84]:
import boto3
import json

# Initialize a session using your AWS credentials
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRECT_ACCESS_KEY,
    region_name='us-east-1'  # e.g., 'us-west-1'
)

# Create an S3 client
s3 = session.client('s3')

# Convert JSON object to string
json_data = json.dumps(output)

# Define your bucket name and the desired object name (key)
bucket_name = 'gz5427'
object_name = 'llm_extract/extracted_Fromweb.json'

# Upload JSON string to S3
s3.put_object(Bucket=bucket_name, Key=object_name, Body=json_data)

print(f"Uploaded {object_name} to {bucket_name}")

Uploaded llm_extract/extracted_Fromweb.json to gz5427


In [49]:
####. Method 1: Use this function to store data into S3.
####.           This is probabaly the easiest way and easy to debug.
def storeintoS3(data, bucketname, filepath):
    import boto3
    import json

    # Initialize a session using your AWS credentials
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRECT_ACCESS_KEY,
        region_name='us-east-1'  # e.g., 'us-west-1'
    )

    # Create an S3 client
    s3 = session.client('s3')
    
    # Convert JSON object to string
    json_data = json.dumps(data)

    # Upload JSON string to S3
    s3.put_object(Bucket=bucketname, Key=filepath, Body=json_data)
    
storeintoS3(data=output, bucketname='gz5427', filepath='llm_extract/testlcel_v3.json')   

In [50]:
####. Method 1: Use the runable to store data into S3.
####.           Right now, this works for just one input!!!! 
def storeintoS3(data):
    import boto3
    import json

    # Initialize a session using your AWS credentials
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRECT_ACCESS_KEY,
        region_name='us-east-1'  # e.g., 'us-west-1'
    )

    # Create an S3 client
    s3 = session.client('s3')
    
    # Convert JSON object to string
    json_data = json.dumps(data)

    # Define your bucket name and the desired object name (key)
    bucket_name = 'gz5427'
    object_name = 'llm_extract/testlcel_v20.json'

    # Upload JSON string to S3
    s3.put_object(Bucket=bucket_name, Key=object_name, Body=json_data)
    

In [51]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

chain = (
    RunnableLambda(lambda x: [{"input": doc} for doc in text_splitter.split_text(x)])
    | extraction_chain.map()
    | flatten
    | storeintoS3
)

In [52]:
tt=chain.invoke(doc.page_content)

In [118]:
####. Method 3:  Pass 3 parameter, yet not working now.
####             Need some debugging if we want to make it work for rua=nnable.

def storeintoS3_passthrough(input):
    import boto3
    import json

    # Initialize a session using your AWS credentials
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRECT_ACCESS_KEY,
        region_name='us-east-1'  # e.g., 'us-west-1'
    )

    # Create an S3 client
    s3 = session.client('s3')
    
    # Convert JSON object to string
    #json_data = json.dumps(input[0])
    json_data = json.dumps(input['data'])

    # Define your bucket name and the desired object name (key)
    bucket_name = input['bucket_name']
    object_name = input['filepath']

    # Upload JSON string to S3
    s3.put_object(Bucket=bucket_name, Key=object_name, Body=json_data)
    

In [119]:
#storeintoS3(data=output, bucketname='gz5427', filepath='llm_extract/output_test.json')

In [120]:
#####. Need some debugging!!!!!!!
#####. Need some debugging!!!!!!!
#####. Need some debugging!!!!!!!
#####. Need some debugging!!!!!!!

chain = (
    RunnableLambda(lambda x: [{"input": doc} for doc in text_splitter.split_text(x)])
    | extraction_chain.map()
    | flatten
    | RunnableParallel(data=RunnablePassthrough(),bucket_name=lambda x: 'gz5427',object_name=lambda x: 'llm_extract/passthrough_testlcel.json'  ) 
    | storeintoS3_passthrough
)

In [112]:
chain

RunnableLambda(lambda x: [{'input': doc} for doc in text_splitter.split_text(x)])
| RunnableEach(bound=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="A article will be passed to you. Extract from it all papers that are mentioned by this article. \n\nDo not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.\n\nDo not make up or guess ANY extra information. Only extract what exactly is in the text."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
  | RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10b16c0e0>, async_client=<openai.resources.chat.completions

In [ ]:
tt2=chain.invoke(doc.page_content)

In [56]:
sts_client = boto3.client('sts')

# Call the GetCallerIdentity operation to retrieve the account ID
response = sts_client.get_caller_identity()
account_id = response['Account']
account_id

'304413155178'

In [57]:
region = boto3.Session().region_name
region

'us-east-1'